###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de https://www.datos.gov.co y corresponden a datos cuya fuente primaria es el Sistema de Información de Eventos de Violencia del Conflicto Armado SIEVCAC

In [ ]:
import pandas as pd

### Descarga de set de datos y carga como df

Se descarga 11 conjuntos de datos que provienen del Sistema de Información de Eventos de Violencia del Conflicto Armado en Colombia (SIEVCAC) y recopilan datos sobre las víctimas del conflicto armado, abarcando distintos tipos de hechos victimizantes. Cada dataset está dedicado a un tipo específico de violencia: desaparición forzada, atentados terroristas, secuestro, asesinatos selectivos, víctimas de minas antipersonal, violencia sexual, daño a bienes civiles, masacres, acciones bélicas, ataques a poblados y reclutamiento de menores

In [ ]:
# Lista urls victimas del conflicto -
victims_dataset_urls = [
     ("victimas_desaparicion_forzada", "https://www.datos.gov.co/resource/c59y-p4sz.csv"),
     ("victimas_atentado_terrorista", "https://www.datos.gov.co/resource/4jfs-zi6u.csv"),
     ("victimas_secuestro", "https://www.datos.gov.co/resource/sg55-sszm.csv"),
     ("victimas_asesinatos_selectivos", "https://www.datos.gov.co/resource/djr9-7s3i.csv"),
     ("victimas_minas", "https://www.datos.gov.co/resource/52eu-ic7d.csv"),
     ("victimas_violencia_sexual", "https://www.datos.gov.co/resource/gtg9-672t.csv"),
     ("victimas_dano_bienes_civiles", "https://www.datos.gov.co/resource/a2ga-ur2i.csv"),
     ("victimas_masacres", "https://www.datos.gov.co/resource/cj8q-zu3u.csv"),
     ("victimas_acciones_belicas", "https://www.datos.gov.co/resource/chb6-bfmq.csv"),
     ("victimas_ataques_a_poblados", "https://www.datos.gov.co/resource/g3r2-mc68.csv"),
     ("victimas_reclutamiento_ninos", "https://www.datos.gov.co/resource/fc55-h9nc.csv"),
 ]

In [ ]:
# Función para descargar datasets y almacenarlos en un diccionario
def download_datasets(victims_dataset_urls):
    datasets = {}
    for name, url in victims_dataset_urls:
        try:
            # Descargar el dataset y cargarlo en un DataFrame
            df = pd.read_csv(url+'?$limit=1000000')
            # Almacenar el DataFrame en el diccionario con el nombre del dataset
            datasets[name] = df
            print(f"Descargado y almacenado en memoria: {name}")
        except Exception as e:
            print(f"Error al descargar {name}: {str(e)}")
    return datasets


In [ ]:
# Llamar a la función y almacenar los datasets en un diccionario
datasets = download_datasets(victims_dataset_urls)

Descargado y almacenado en memoria: victimas_desaparicion_forzada
Descargado y almacenado en memoria: victimas_atentado_terrorista
Descargado y almacenado en memoria: victimas_secuestro
Descargado y almacenado en memoria: victimas_asesinatos_selectivos
Descargado y almacenado en memoria: victimas_minas
Descargado y almacenado en memoria: victimas_violencia_sexual
Descargado y almacenado en memoria: victimas_dano_bienes_civiles
Descargado y almacenado en memoria: victimas_masacres
Descargado y almacenado en memoria: victimas_acciones_belicas
Descargado y almacenado en memoria: victimas_ataques_a_poblados
Descargado y almacenado en memoria: victimas_reclutamiento_ninos


In [ ]:
# Función para agregar una columna 'tipo_de_crimen' a cada dataset
def add_type_of_crime(datasets):
    for name, df in datasets.items():
        # Remover 'victimas_' del nombre del dataset
        tipo_crimen = name.replace('victimas_', '')
        # Agregar la nueva columna al DataFrame
        df['tipo_de_crimen'] = tipo_crimen
        # Actualizar el DataFrame en el diccionario
        datasets[name] = df

    return datasets

In [ ]:
# Ejecutar funcion add_type_of_crime
datasets = add_type_of_crime(datasets)

# Verificar que la columna fue agregada en un dataset
datasets['victimas_asesinatos_selectivos'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181048 entries, 0 to 181047
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   id_caso                       181048 non-null  int64 
 1   c_digo_dane_de_municipio      181048 non-null  int64 
 2   municipio                     181048 non-null  object
 3   departamento                  181048 non-null  object
 4   a_o                           181048 non-null  int64 
 5   mes                           181048 non-null  int64 
 6   d_a                           181048 non-null  int64 
 7   id_persona                    181048 non-null  int64 
 8   sexo                          181048 non-null  object
 9   etnia                         3029 non-null    object
 10  ocupaci_n                     181048 non-null  object
 11  calidad_de_la_v_ctima_o_la    181048 non-null  object
 12  tipo_de_poblaci_n_vulnerable  19620 non-null   object
 13 

In [ ]:
# Función para obtener las columnas comunes a todos los datasets
def common_columns(datasets):
    # Extraer las columnas del primer dataset como base
    common_columns = set(datasets[next(iter(datasets))].columns)

    # Iterar sobre los demás datasets y realizar la intersección de columnas
    for name, df in datasets.items():
        common_columns &= set(df.columns)  # Intersección de conjuntos

    return list(common_columns)  # Retornar el resultado como lista

In [ ]:
# Obtener las columnas comunes a todos los datasets
all_common_columns = common_columns(datasets)

# Mostrar las columnas comunes
print("Columnas comunes a todos los datasets:", all_common_columns)

Columnas comunes a todos los datasets: ['id_persona', 'd_a', 'tipo_de_crimen', 'id_caso', 'c_digo_dane_de_municipio', 'departamento', 'ocupaci_n', 'a_o', 'latitud_longitud', 'sexo', 'mes', 'edad', 'etnia', 'municipio']


In [ ]:
# Funcion para filtar solamente columnas comunes
def filter_common_columns(datasets, common_columns):
    for name, df in datasets.items():
        # Filtrar el DataFrame para que solo contenga las columnas en common_columns
        datasets[name] = df.filter(items=common_columns)
    return datasets

In [ ]:
# Ejemplo de uso
filtered_datasets = filter_common_columns(datasets, all_common_columns)

# Verificar que los DataFrames han sido filtrados
for name, df in filtered_datasets.items():
    print(f"\n{ name }:\n", df.columns)


victimas_desaparicion_forzada:
 Index(['id_persona', 'd_a', 'tipo_de_crimen', 'id_caso',
       'c_digo_dane_de_municipio', 'departamento', 'ocupaci_n', 'a_o',
       'latitud_longitud', 'sexo', 'mes', 'edad', 'etnia', 'municipio'],
      dtype='object')

victimas_atentado_terrorista:
 Index(['id_persona', 'd_a', 'tipo_de_crimen', 'id_caso',
       'c_digo_dane_de_municipio', 'departamento', 'ocupaci_n', 'a_o',
       'latitud_longitud', 'sexo', 'mes', 'edad', 'etnia', 'municipio'],
      dtype='object')

victimas_secuestro:
 Index(['id_persona', 'd_a', 'tipo_de_crimen', 'id_caso',
       'c_digo_dane_de_municipio', 'departamento', 'ocupaci_n', 'a_o',
       'latitud_longitud', 'sexo', 'mes', 'edad', 'etnia', 'municipio'],
      dtype='object')

victimas_asesinatos_selectivos:
 Index(['id_persona', 'd_a', 'tipo_de_crimen', 'id_caso',
       'c_digo_dane_de_municipio', 'departamento', 'ocupaci_n', 'a_o',
       'latitud_longitud', 'sexo', 'mes', 'edad', 'etnia', 'municipio'],
      dty

In [ ]:
# Integrar verticalmente todos los datasets
# Integrar todos los DataFrames filtrados verticalmente
all_datasets = pd.concat(filtered_datasets.values(), ignore_index=True)

# Verificar la estructura del DataFrame integrado
all_datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420988 entries, 0 to 420987
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   id_persona                420988 non-null  int64 
 1   d_a                       420988 non-null  int64 
 2   tipo_de_crimen            420988 non-null  object
 3   id_caso                   420988 non-null  int64 
 4   c_digo_dane_de_municipio  420988 non-null  int64 
 5   departamento              420988 non-null  object
 6   ocupaci_n                 420987 non-null  object
 7   a_o                       420988 non-null  int64 
 8   latitud_longitud          420988 non-null  object
 9   sexo                      420988 non-null  object
 10  mes                       420988 non-null  int64 
 11  edad                      189310 non-null  object
 12  etnia                     9411 non-null    object
 13  municipio                 420988 non-null  object
dtypes: i

In [ ]:
# Ajuste nombre dataset
victims_armed_conflict_sievcac = all_datasets.copy()
victims_armed_conflict_sievcac.head()

,id_persona,d_a,tipo_de_crimen,id_caso,c_digo_dane_de_municipio,departamento,ocupaci_n,a_o,latitud_longitud,sexo,mes,edad,etnia,municipio
0,72226,25,desaparicion_forzada,100265,25368,CUNDINAMARCA,PERSONAL DE SALUD,1991,POINT (-74.695419232 4.5623082672),HOMBRE,5,NaN,NaN,JERUSALEN
1,129425,2,desaparicion_forzada,100282,25386,CUNDINAMARCA,EMPLEADO,2004,POINT (-74.4615447441 4.631044239),HOMBRE,12,ADULTEZ (29-59 AÑOS),NaN,LA MESA
2,71898,9,desaparicion_forzada,101616,25885,CUNDINAMARCA,CAMPESINO,1993,POINT (-74.3380243467 5.4592903412),HOMBRE,3,NaN,NaN,YACOPI
3,71900,9,desaparicion_forzada,101616,25885,CUNDINAMARCA,CAMPESINO,1993,POINT (-74.3380243467 5.4592903412),HOMBRE,3,ADOLESCENCIA (12-17 AÑOS),NaN,YACOPI
4,80640,15,desaparicion_forzada,102675,27615,CHOCO,SIN INFORMACIÓN,1997,POINT (-77.1131095942 7.4366315522),HOMBRE,5,ADULTEZ (29-59 AÑOS),NaN,RIOSUCIO


In [ ]:
columns = victims_armed_conflict_sievcac.columns

In [ ]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {victims_armed_conflict_sievcac[col].nunique()}')

id_persona: 420178
d_a: 32
tipo_de_crimen: 11
id_caso: 321222
c_digo_dane_de_municipio: 1148
departamento: 35
ocupaci_n: 34
a_o: 66
latitud_longitud: 5018
sexo: 3
mes: 13
edad: 6
etnia: 5
municipio: 1037


In [ ]:
# Eliminar columnas que se considera innecesarias para el proyecto
important_columns = ['id_caso', 'departamento', 'municipio', 'c_digo_dane_de_municipio','a_o', 'tipo_de_crimen', 'sexo', 'edad',  'etnia', 'ocupaci_n', ]
victims_armed_conflict_sievcac = victims_armed_conflict_sievcac[important_columns]
victims_armed_conflict_sievcac.head()

,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,sexo,edad,etnia,ocupaci_n
0,100265,CUNDINAMARCA,JERUSALEN,25368,1991,desaparicion_forzada,HOMBRE,NaN,NaN,PERSONAL DE SALUD
1,100282,CUNDINAMARCA,LA MESA,25386,2004,desaparicion_forzada,HOMBRE,ADULTEZ (29-59 AÑOS),NaN,EMPLEADO
2,101616,CUNDINAMARCA,YACOPI,25885,1993,desaparicion_forzada,HOMBRE,NaN,NaN,CAMPESINO
3,101616,CUNDINAMARCA,YACOPI,25885,1993,desaparicion_forzada,HOMBRE,ADOLESCENCIA (12-17 AÑOS),NaN,CAMPESINO
4,102675,CHOCO,RIOSUCIO,27615,1997,desaparicion_forzada,HOMBRE,ADULTEZ (29-59 AÑOS),NaN,SIN INFORMACIÓN


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
victims_armed_conflict_sievcac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420988 entries, 0 to 420987
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   id_caso                   420988 non-null  int64 
 1   departamento              420988 non-null  object
 2   municipio                 420988 non-null  object
 3   c_digo_dane_de_municipio  420988 non-null  int64 
 4   a_o                       420988 non-null  int64 
 5   tipo_de_crimen            420988 non-null  object
 6   sexo                      420988 non-null  object
 7   edad                      189310 non-null  object
 8   etnia                     9411 non-null    object
 9   ocupaci_n                 420987 non-null  object
dtypes: int64(3), object(7)
memory usage: 32.1+ MB


In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'tipo_de_crimen', 'sexo', 'edad', 'etnia','ocupaci_n']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(victims_armed_conflict_sievcac[column].unique())
    print()

Categorías en la columna 'departamento':
['CUNDINAMARCA' 'CHOCO' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'SANTANDER' 'SUCRE' 'TOLIMA' 'CASANARE'
 'VALLE DEL CAUCA' 'PUTUMAYO' 'ANTIOQUIA' 'BOLIVAR' 'CAQUETA' 'CAUCA'
 'CESAR' 'CORDOBA' 'CALDAS' 'EXTERIOR' 'GUAVIARE' 'ARAUCA' 'RISARALDA'
 'QUINDIO' 'BOYACA' 'BOGOTA, D. C.' 'ATLANTICO' 'SIN INFORMACION'
 'VICHADA' 'VAUPES' 'GUAINIA' 'AMAZONAS'
 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA']

Categorías en la columna 'municipio':
['JERUSALEN' 'LA MESA' 'YACOPI' ... 'FRONTERA PERÚ' 'PUERTO NARIÑO'
 'GACHANTIVA']

Categorías en la columna 'tipo_de_crimen':
['desaparicion_forzada' 'atentado_terrorista' 'secuestro'
 'asesinatos_selectivos' 'minas' 'violencia_sexual' 'dano_bienes_civiles'
 'masacres' 'acciones_belicas' 'ataques_a_poblados' 'reclutamiento_ninos']

Categorías en la columna 'sexo':
['HOMBRE' 'MUJER' 'SIN INFORMACION']

Categorías en la columna 'edad':
[nan 'ADULTEZ (29-59 AÑOS)' 'ADOLESCE

In [ ]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in categorical_col:
  l = victims_armed_conflict_sievcac[col].nunique()
  if l < 50:
    print(f'{col}: {victims_armed_conflict_sievcac[col].unique()}')

departamento: ['CUNDINAMARCA' 'CHOCO' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'SANTANDER' 'SUCRE' 'TOLIMA' 'CASANARE'
 'VALLE DEL CAUCA' 'PUTUMAYO' 'ANTIOQUIA' 'BOLIVAR' 'CAQUETA' 'CAUCA'
 'CESAR' 'CORDOBA' 'CALDAS' 'EXTERIOR' 'GUAVIARE' 'ARAUCA' 'RISARALDA'
 'QUINDIO' 'BOYACA' 'BOGOTA, D. C.' 'ATLANTICO' 'SIN INFORMACION'
 'VICHADA' 'VAUPES' 'GUAINIA' 'AMAZONAS'
 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA']
tipo_de_crimen: ['desaparicion_forzada' 'atentado_terrorista' 'secuestro'
 'asesinatos_selectivos' 'minas' 'violencia_sexual' 'dano_bienes_civiles'
 'masacres' 'acciones_belicas' 'ataques_a_poblados' 'reclutamiento_ninos']
sexo: ['HOMBRE' 'MUJER' 'SIN INFORMACION']
edad: [nan 'ADULTEZ (29-59 AÑOS)' 'ADOLESCENCIA (12-17 AÑOS)'
 'JUVENTUD (18-28 AÑOS)' 'PERSONA MAYOR (60 AÑOS O MAS)'
 'INFANCIA (6-11 AÑOS)' 'PRIMERA INFANCIA (0-5 AÑOS)']
etnia: [nan 'RAIZAL' 'INDÍGENA' 'AFROCOLOMBIANO' 'PALENQUERO' 'ROM']
ocupaci_n: ['PERSONAL DE SALUD' '

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
victims_armed_conflict_sievcac[categorical_col] = victims_armed_conflict_sievcac[categorical_col].astype(str)
for col in categorical_col:
    victims_armed_conflict_sievcac[col] = victims_armed_conflict_sievcac[col].apply(remove_accents_and_special_chars)

In [ ]:
# Columna edad

# Diccionario de mapeo de categorías
mapping = {
    'ADULTEZ (29-59 ANOS)': 'ADULTEZ',
    'ADOLESCENCIA (12-17 ANOS)': 'ADOLESCENCIA',
    'JUVENTUD (18-28 ANOS)': 'JUVENTUD',
    'PERSONA MAYOR (60 ANOS O MAS)': 'PERSONA MAYOR',
    'INFANCIA (6-11 ANOS)': 'INFANCIA',
    'PRIMERA INFANCIA (0-5 ANOS)': 'PRIMERA INFANCIA'
}

# Aplicar el mapeo de las categorías
victims_armed_conflict_sievcac['edad'] = victims_armed_conflict_sievcac['edad'].map(mapping)

# Reemplazar los valores NaN con 'DESCONOCIDA' sin usar inplace
victims_armed_conflict_sievcac['edad'] = victims_armed_conflict_sievcac['edad'].fillna('DESCONOCIDA')
victims_armed_conflict_sievcac['edad'].unique()

array(['DESCONOCIDA', 'ADULTEZ', 'ADOLESCENCIA', 'JUVENTUD',
       'PERSONA MAYOR', 'INFANCIA', 'PRIMERA INFANCIA'], dtype=object)

In [ ]:
# Columna sex

# Reemplazar 'SIN INFORMACION' por 'DESCONOCIDO'
victims_armed_conflict_sievcac['sexo'].replace('SIN INFORMACION', 'DESCONOCIDO', inplace=True)
victims_armed_conflict_sievcac['sexo'].unique()

<ipython-input-22-9ab5f3346ac4>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  victims_armed_conflict_sievcac['sexo'].replace('SIN INFORMACION', 'DESCONOCIDO', inplace=True)


array(['HOMBRE', 'MUJER', 'DESCONOCIDO'], dtype=object)

In [ ]:
# Variable etnia
import numpy as np
victims_armed_conflict_sievcac['etnia'].replace('NAN', np.nan, inplace=True)
victims_armed_conflict_sievcac['etnia'] = victims_armed_conflict_sievcac['etnia'].fillna('DESCONOCIDA')

# Variable ocupacion
victims_armed_conflict_sievcac['ocupaci_n'].replace({
    'OTRA ¿CUAL': 'OTRA',
    'SIN INFORMACION': 'DESCONOCIDA',
    'TRABAJADOR(A) SEXUAL': 'TRABAJADOR SEXUAL',
    'NO APLICA': 'DESCONOCIDA',
}, inplace=True)

# Reemplazar NaN por 'DESCONOCIDA'
victims_armed_conflict_sievcac['ocupaci_n'].fillna('DESCONOCIDA')

print(victims_armed_conflict_sievcac['etnia'].unique())
victims_armed_conflict_sievcac['ocupaci_n'].unique()

<ipython-input-23-04e0d0789076>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  victims_armed_conflict_sievcac['etnia'].replace('NAN', np.nan, inplace=True)
<ipython-input-23-04e0d0789076>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

['DESCONOCIDA' 'RAIZAL' 'INDIGENA' 'AFROCOLOMBIANO' 'PALENQUERO' 'ROM']


array(['PERSONAL DE SALUD', 'EMPLEADO', 'CAMPESINO', 'DESCONOCIDA',
       'PARAMILITAR', 'DELINCUENTE', 'DESEMPLEADO', 'TRABAJADOR DE FINCA',
       'GANADERO/HACENDADO', 'CONDUCTOR/MOTORISTA', 'ECONOMIA INFORMAL',
       'COMERCIANTE', 'OTRA', 'ESTUDIANTE', 'OBRERO',
       'TRABAJO SIN ESPECIFICAR', 'ADMINISTRADOR DE FINCA',
       'TRABAJADOR SEXUAL', 'FUERZA PUBLICA', 'SEGURIDAD PRIVADA',
       'RELIGIOSO', 'GUERRILLERO', 'PROFESIONAL', 'PESCADOR', 'RASPACHIN',
       'MINERO', 'FUNCIONARIO PUBLICO', 'PENSIONADO', 'AMA DE CASA',
       'EMPRESARIO - INDUSTRIAL', 'MIEMBRO DE GRUPO POSDESMOVILIZACION',
       'NAN', 'BANDOLERO', 'ERRADICADOR'], dtype=object)

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios estan en formato int64


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [ ]:
victims_armed_conflict_sievcac.columns

Index(['id_caso', 'departamento', 'municipio', 'c_digo_dane_de_municipio',
       'a_o', 'tipo_de_crimen', 'sexo', 'edad', 'etnia', 'ocupaci_n'],
      dtype='object')

In [ ]:
# valores codigo dane del municipio
victims_armed_conflict_sievcac["c_digo_dane_de_municipio"].unique()

array([25368, 25386, 25885, ..., 91540, 15293, 97000])

- Longitud de los códigos municipios en el df

In [ ]:
# Convertir a strings
victims_armed_conflict_sievcac["c_digo_dane_de_municipio"] = victims_armed_conflict_sievcac["c_digo_dane_de_municipio"].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_mpio = victims_armed_conflict_sievcac["c_digo_dane_de_municipio"].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los mpios tiene longitudes de :[5 4 1 6] caracteres


In [ ]:
# Contar registros por longitud
long_df = longitud_mpio.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitud_mpio))
long_df.head()

,c_digo_dane_de_municipio,count,percentage
0,5,291710,0.692918
1,4,119888,0.284778
2,1,9389,0.022302
3,6,1,0.000002


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitud_mpio.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = victims_armed_conflict_sievcac[longitud_mpio == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['c_digo_dane_de_municipio']])
    print()

Muestra de registros con longitud 5:
  c_digo_dane_de_municipio
0                    25368
1                    25386
2                    25885
3                    25885
4                    27615

Muestra de registros con longitud 4:
   c_digo_dane_de_municipio
80                     5361
81                     5361
82                     5480
83                     5480
84                     5480

Muestra de registros con longitud 1:
    c_digo_dane_de_municipio
452                        0
453                        0
454                        0
455                        0
456                        0

Muestra de registros con longitud 6:
      c_digo_dane_de_municipio
74076                   770000



Nota: De lo anterior se notan claras inconsistencias en la columna 'c_digo_dane_de_municipio' del df, el 69.3% tiene 5 dígitos (que corresponde con el codigo real que es de 5 dígitos), el 28.5% tiene solamente 4 dígitos (al parecer el cero a la izquierda de los códigos se suprimió), un 2.23% tienen un codigo de un digito, que al parecer es el 0, y existe uno registro con un codigo de 6 digitos: 770000

- Verificar si los codigo_dane de 4 dígitos corresponden a departamentos que se identifican con el 0 + 1 dígito, para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [ ]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 4]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane que inicia con un cero + un dígito, 05 y 08 respectivamente.

  - Adicionar un cero a los codigo_dane de 4 dígitos

In [ ]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
victims_armed_conflict_sievcac['c_digo_dane_de_municipio'] = victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].apply(add_zero_if_length_4)

In [ ]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 4])

0

- Resolver caso de codigo de 6 digitos

In [ ]:
# Ver registro
victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 6]

,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,sexo,edad,etnia,ocupaci_n
74076,434643,EXTERIOR,FRONTERA,770000,2003,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA


In [ ]:
# Se elimina registro porque caso no sucedio en Colombia
victims_armed_conflict_sievcac = victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() != 6]

- Resolver caso con codigo de 1 digito

In [ ]:
# Imprimir una muestra
code_1_digit = victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 1]
code_1_digit.head()

,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,sexo,edad,etnia,ocupaci_n
452,205568,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,CAMPESINO
453,205569,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,MUJER,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA
454,205570,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,COMERCIANTE
455,205571,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA
456,205572,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA


In [ ]:
# Verificar ninguno cuenta con informacion de departamento y municipio
print(code_1_digit['departamento'].unique())
print(code_1_digit['municipio'].unique())
print(code_1_digit['edad'].unique())
print(code_1_digit['etnia'].unique())

['SIN INFORMACION']
['SIN INFORMACION']
['DESCONOCIDA' 'JUVENTUD' 'ADOLESCENCIA' 'ADULTEZ' 'INFANCIA'
 'PRIMERA INFANCIA' 'PERSONA MAYOR']
['DESCONOCIDA' 'AFROCOLOMBIANO' 'INDIGENA']


In [ ]:
# Se decide elimnar dichos registros porque son casos sin informacion, y no existe posibilidad de actualizarlos
victims_armed_conflict_sievcac = victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() != 1]

In [ ]:
victims_armed_conflict_sievcac.shape

(411598, 10)

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [ ]:
# Comparar listas de códigos
compare_lists(victims_armed_conflict_sievcac['c_digo_dane_de_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_armed_conflict_sievcac", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_armed_conflict_sievcac que no están en Códigos de municipios dept_mpios_codes:
{'77003', '86000', '20000', '94663', '76000', '08000', '88000', '77005', '44000', '63000', '77000', '17000', '91000', '25000', '85000', '13000', '27000', '81000', '50000', '27086', '94000', '47000', '77001', '73000', '70000', '15000', '66000', '23000', '18000', '97000', '77004', '68000', '19000', '95000', '11000', '54000', '99000', '41000', '77002', '52000', '05000'}


In [ ]:
# codigos que no corresponden a codigos reales de municipios
codes_wront = {'81000', '95000', '15000', '70000', '50000', '76000', '77000', '41000', '99000', '05000', '68000', '27086', '63000', '54000', '20000', '91000', '27000', '52000', '66000', '85000', '13000', '97000', '23000', '77002', '86000', '77001', '77005', '73000', '94663', '19000', '94000', '77004', '08000', '18000', '88000', '25000', '44000', '47000', '17000', '11000', '77003'}

In [ ]:
# Imprimir una muestra
registers_confirm = victims_armed_conflict_sievcac[victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].isin(codes_wront)]
print(f'NO. registros con codigo errado: {len(registers_confirm)}')
registers_confirm.head()

NO. registros con codigo errado: 3050


,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,sexo,edad,etnia,ocupaci_n
125,205166,EXTERIOR,FRONTERA PANAMA,77003,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA
126,205167,EXTERIOR,FRONTERA PANAMA,77003,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA
459,205577,ANTIOQUIA,SIN INFORMACION,05000,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA
460,205579,CAQUETA,SIN INFORMACION,18000,0,DESAPARICION_FORZADA,MUJER,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA
461,205580,CAUCA,SIN INFORMACION,19000,0,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,DESCONOCIDA


In [ ]:
# Verificamos categoria de municipios con codigos errados
registers_confirm['municipio'].value_counts()

,count
municipio,
SIN INFORMACION,2593
FRONTERA VENEZUELA,280
FRONTERA ECUADOR,97
FRONTERA PANAMA,33
MAPIRIPANA,25
BELEN DE BAJIRA,8
FRONTERA PERU,6
FRONTERA BRASIL,5
FRONTERA,3


Nota: No existe informacion sobre el municipio o corresponden a hechos fuera de colombia, salvo  Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio Barranco Minas de Guania, y Belen de Bajira, que en la bd figura como corregimiento de Mutata. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece y borramos los registros que corresponden al exterior y los que no se tiene informacion

In [ ]:
# Verificar el registro con que aparecen
maripana = registers_confirm[registers_confirm['municipio']=='MAPIRIPANA']
print(maripana['c_digo_dane_de_municipio'].unique())


belen = registers_confirm[registers_confirm['municipio']=='BELEN DE BAJIRA']
print(belen['c_digo_dane_de_municipio'].unique())

['94663']
['27086']


In [ ]:
# Buscamos los codigos de los municipios a los que pertenecen
print(dept_mpios_codes[dept_mpios_codes['mupio_name'] == 'BARRANCOMINAS'])
dept_mpios_codes[dept_mpios_codes['mupio_name'] == 'MUTATA']

     dept_mpio_code dept_name     mupio_name
1093          94343   GUAINIA  BARRANCOMINAS


,dept_mpio_code,dept_name,mupio_name
77,05480,ANTIOQUIA,MUTATA


In [ ]:
# Reemplazar
victims_armed_conflict_sievcac['c_digo_dane_de_municipio'] = victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].replace( '94663', '94343')
victims_armed_conflict_sievcac['c_digo_dane_de_municipio'] = victims_armed_conflict_sievcac['c_digo_dane_de_municipio'].replace( '27086', '05480')

In [ ]:
# Eliminamos los otros registros que corresponden al exterior y los sin informacion
# Definir las categorías a eliminar
delete_cat = ['SIN INFORMACION', 'FRONTERA VENEZUELA','FRONTERA ECUADOR','FRONTERA PANAMA','FRONTERA PERU','FRONTERA BRASIL','FRONTERA']

# Filtrar el DataFrame para eliminar las categorías especificadas
victims_armed_conflict_sievcac = victims_armed_conflict_sievcac[~victims_armed_conflict_sievcac['municipio'].isin(delete_cat)]

(408581, 10)

- Volvemos a veridfcar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [ ]:
# Comparar listas de códigos
compare_lists(victims_armed_conflict_sievcac['c_digo_dane_de_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_armed_conflict_sievcac", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_armed_conflict_sievcac que no están en Códigos de municipios dept_mpios_codes:
set()


In [ ]:
# Eliminar columnas 'departamento' y 'municipio' que ya no son necesarias
victims_armed_conflict_sievcac = victims_armed_conflict_sievcac.drop(['departamento', 'municipio'], axis=1)
victims_armed_conflict_sievcac.head(2)

,id_caso,c_digo_dane_de_municipio,a_o,tipo_de_crimen,sexo,edad,etnia,ocupaci_n
0,100265,25368,1991,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,PERSONAL DE SALUD
1,100282,25386,2004,DESAPARICION_FORZADA,HOMBRE,ADULTEZ,DESCONOCIDA,EMPLEADO


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
victims_armed_conflict_sievcac['tipo_de_crimen'].unique()

array(['DESAPARICION_FORZADA', 'ATENTADO_TERRORISTA', 'SECUESTRO',
       'ASESINATOS_SELECTIVOS', 'MINAS', 'VIOLENCIA_SEXUAL',
       'DANO_BIENES_CIVILES', 'MASACRES', 'ACCIONES_BELICAS',
       'ATAQUES_A_POBLADOS', 'RECLUTAMIENTO_NINOS'], dtype=object)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
# Lista de tuplas
sources_id = [
    ('DESAPARICION_FORZADA', 87),
    ('ATENTADO_TERRORISTA', 88),
    ('SECUESTRO', 89),
    ('ASESINATOS_SELECTIVOS', 90),
    ('MINAS', 91),
    ('VIOLENCIA_SEXUAL', 92),
    ('DANO_BIENES_CIVILES', 93),
    ('MASACRES', 94),
    ('ACCIONES_BELICAS', 95),
    ('ATAQUES_A_POBLADOS', 96),
    ('RECLUTAMIENTO_NINOS', 97)
]

# Convertir la lista de tuplas en un diccionario para facilitar el mapeo
sources_dict = dict(sources_id)

# Crear la columna 'source_id' asignando el valor condicionalmente
victims_armed_conflict_sievcac['source_id'] = victims_armed_conflict_sievcac['tipo_de_crimen'].map(sources_dict)


In [ ]:
# Agrupar el DataFrame por la columna 'source_id' y obtener el primer elemento de cada grupo
proof = victims_armed_conflict_sievcac.groupby('source_id').first()

# Mostrar los resultados
print(proof)

           id_caso c_digo_dane_de_municipio   a_o         tipo_de_crimen  \
source_id                                                                  
87          100265                    25368  1991   DESAPARICION_FORZADA   
88          185269                    17001  1991    ATENTADO_TERRORISTA   
89          102517                    27001  2005              SECUESTRO   
90          100000                    25290  2003  ASESINATOS_SELECTIVOS   
91          113698                    50325  2012                  MINAS   
92          235240                    17614  1997       VIOLENCIA_SEXUAL   
93          162442                    23660  1990    DANO_BIENES_CIVILES   
94          100649                    25530  1998               MASACRES   
95               1                    27073  2006       ACCIONES_BELICAS   
96           10902                    27073  1997     ATAQUES_A_POBLADOS   
97          207556                    05756  2001    RECLUTAMIENTO_NINOS   

           

In [ ]:
victims_armed_conflict_sievcac.columns

Index(['id_caso', 'c_digo_dane_de_municipio', 'a_o', 'tipo_de_crimen', 'sexo',
       'edad', 'etnia', 'ocupaci_n', 'source_id'],
      dtype='object')

In [ ]:
# Crear el diccionario manualmente
column_mapping = {
    'id_caso': 'id_sievcac',
    'c_digo_dane_de_municipio': 'dane_code',
    'a_o': 'year',
    'tipo_de_crimen': 'type_of_crime',
    'sexo': 'sex',
    'edad': 'age',
    'etnia': 'ethnic_group',
    'ocupaci_n': 'ocupation',
    'source_id': 'source_id'
}

# Renombrar las columnas del DataFrame
victims_armed_conflict_sievcac = victims_armed_conflict_sievcac.rename(columns=column_mapping)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
victims_armed_conflict_sievcac.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408581 entries, 0 to 420987
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id_sievcac     408581 non-null  int64 
 1   dane_code      408581 non-null  object
 2   year           408581 non-null  int64 
 3   type_of_crime  408581 non-null  object
 4   sex            408581 non-null  object
 5   age            408581 non-null  object
 6   ethnic_group   408581 non-null  object
 7   ocupation      408581 non-null  object
 8   source_id      408581 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 31.2+ MB


In [ ]:
victims_armed_conflict_sievcac.head()

,id_sievcac,dane_code,year,type_of_crime,sex,age,ethnic_group,ocupation,source_id
0,100265,25368,1991,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,PERSONAL DE SALUD,87
1,100282,25386,2004,DESAPARICION_FORZADA,HOMBRE,ADULTEZ,DESCONOCIDA,EMPLEADO,87
2,101616,25885,1993,DESAPARICION_FORZADA,HOMBRE,DESCONOCIDA,DESCONOCIDA,CAMPESINO,87
3,101616,25885,1993,DESAPARICION_FORZADA,HOMBRE,ADOLESCENCIA,DESCONOCIDA,CAMPESINO,87
4,102675,27615,1997,DESAPARICION_FORZADA,HOMBRE,ADULTEZ,DESCONOCIDA,DESCONOCIDA,87


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
victims_armed_conflict_sievcac.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/victims_armed_conflict_sievcac.csv', index=False)